In [ ]:
# Import the Pandas library and declare the path to the Dados_Candidatos file to start the ETL process.
import pandas as pd

# Insert the path to the Dados_Candidatos.xlsx file. (Ex: D:\Desafio_ISOps\Dados_Candidatos.xlsx)
PATH_TO_DADOS_CANDIDATOS_FILE = ''

# 1. Reading and setting some parameters to avoid false positives when transforming data.

DATA = pd.read_excel(PATH_TO_DADOS_CANDIDATOS_FILE)
DATA.is_copy = False
pd.options.mode.chained_assignment = None

In [ ]:
# 2. Transposing rows of 'Etapa' into distinct columns.

transposed_data = DATA.fillna(value=0).pivot_table(index=['ID_Candidato', 'Nome_Candidato', 'ID_Vaga', 'Nome_Vaga', 
                                                          'Data_Candidatura', 'Status', 'Motivo_Reprovação'],
                                                    columns='Etapa',
                                                    values='Data_Movimentacao',
                                                    aggfunc='last').reset_index()

In [ ]:
# 3. Filter out duplicate dates for a potential merge.

'''
    Create a separate column with the registration date to select the last value and apply a merge on the main table.
'''

registration_data = DATA[DATA['Etapa'] == 'Cadastro'][['ID_Candidato', 'Data_Movimentacao']].rename(columns={'Data_Movimentacao': 'Cadastro'})
transposed_data = pd.merge(transposed_data, registration_data, on='ID_Candidato', how='right')

In [ ]:
# 4. Aggregate all values from the created column to the main column.

transposed_data.loc[~transposed_data['Cadastro_x'].isin(transposed_data['Cadastro_y']), 'Cadastro_x'] = transposed_data['Cadastro_y']

In [ ]:
# 5. Removal of duplicate values -> Renaming the Cadastro_x column back to its original name (Cadastro) -> Removing unnecessary extra indices.

transposed_data = transposed_data.drop_duplicates(subset=['ID_Candidato'], keep='last'
                                                  ).rename(columns={'Cadastro_x':'Cadastro'}
                                                          ).drop(columns=['Cadastro_y']
                                                                ).reset_index(drop=True)

In [ ]:
# 6. Remapping of columns (supposed chronological order).

ordered_columns = ['ID_Candidato', 'Nome_Candidato', 'ID_Vaga', 'Nome_Vaga', 'Data_Candidatura', 
                   'Status', 'Cadastro', 'Triagem', 'Entrevista TA', 'Teste Técnico', 
                   'Entrevista RH', 'Entrevista Final', 'Oferta', 'Contratação', 'Motivo_Reprovação']

ordered_data = transposed_data[ordered_columns].reset_index(drop=True)

In [ ]:
'''
7. Replacement of the 0s assigned in step 2 with null values and 
formatting of the date columns to meet the established criterion: ISO8601 formatting
'''

'''
The check_cell_data_type function is used to verify if the data type of a given cell in a specific column
is in the datetime64[ns] format (the standard Excel format for Date type cells).

Possible outcomes:
True: The correct value to meet the ISO8601 formatting criterion is assigned to the cell.
False: A null value (NA) is assigned to the cell.
'''

def check_cell_data_type(cell_value):
    if pd.api.types.is_datetime64_any_dtype(cell_value):
        return pd.NA if pd.isna(cell_value) else cell_value.date()
    else:
        return cell_value

for column in ordered_data.columns[0:]:
    ordered_data[column] = ordered_data[column].replace(0, pd.NA).apply(check_cell_data_type)

In [ ]:
# 8. Export the final dataframe to a CSV file.

ordered_data.to_csv('Clean and Structured Data.csv', index=False, encoding='utf-8-sig', date_format="%Y-%m-%d")